<a href="https://colab.research.google.com/github/keysenelvial/dinam/blob/main/dinam3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №3**

Провести классификацию найденного датасета, методами линеной и логистической регрессий . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score


data = pd.read_csv('/content/drive/MyDrive/train 3.csv', on_bad_lines='skip')
print(data.head())


# Предобработка данных
data = data[['log_price', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'property_type']].dropna()

# Уменьшение размера датасета: выбор 15% случайных строк
sampled_data = data.sample(frac=0.15, random_state=42)

# Разделение цены на категории
sampled_data['price_category'] = pd.qcut(sampled_data['log_price'], q=4, labels=False)

# Подготовка признаков и целевой переменной
X = sampled_data[['accommodates', 'bathrooms', 'bedrooms', 'beds']]
y = sampled_data['price_category']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



         id  log_price property_type        room_type  \
0   6901257   5.010635     Apartment  Entire home/apt   
1   6304928   5.129899     Apartment  Entire home/apt   
2   7919400   4.976734     Apartment  Entire home/apt   
3  13418779   6.620073         House  Entire home/apt   
4   3808709   4.744932     Apartment  Entire home/apt   

                                           amenities  accommodates  bathrooms  \
0  {"Wireless Internet","Air conditioning",Kitche...             3        1.0   
1  {"Wireless Internet","Air conditioning",Kitche...             7        1.0   
2  {TV,"Cable TV","Wireless Internet","Air condit...             5        1.0   
3  {TV,"Cable TV",Internet,"Wireless Internet",Ki...             4        1.0   
4  {TV,Internet,"Wireless Internet","Air conditio...             2        1.0   

   bed_type cancellation_policy  cleaning_fee  ...   latitude   longitude  \
0  Real Bed              strict          True  ...  40.696524  -73.991617   
1  Real Bed     

In [13]:
# Создание пайплайна для логистической регрессии
logistic_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic', LogisticRegression())
])

# Параметры для GridSearchCV для логистической регрессии
logistic_params = {
    'logistic__C': np.logspace(-4, 4, 20),
    'logistic__solver': ['liblinear', 'saga']  # хорошо подходят для маленьких датасетов
}

# Использование стратифицированной кросс-валидации
cv_strategy = StratifiedKFold(n_splits=5)

logistic_grid_search = GridSearchCV(logistic_pipe, param_grid=logistic_params, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
logistic_grid_search.fit(X_train, y_train)

print('Лучшие параметры (логистическая регрессия):', logistic_grid_search.best_params_)
print('Лучшая оценка:', logistic_grid_search.best_score_)

Лучшие параметры (логистическая регрессия): {'logistic__C': 0.23357214690901212, 'logistic__solver': 'liblinear'}
Лучшая оценка: 0.47819110138584986


In [14]:
# Создание пайплайна для линейной регрессии
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', RidgeClassifier())
])

# Параметры для GridSearchCV для линейной регрессии
ridge_params = {
    'ridge__alpha': np.logspace(-4, 4, 20)
}

ridge_grid_search = GridSearchCV(ridge_pipe, param_grid=ridge_params, cv=5, scoring='accuracy')
ridge_grid_search.fit(X_train, y_train)

print('Лучшие параметры (линейная регрессия):', ridge_grid_search.best_params_)
print('Лучшая оценка:', ridge_grid_search.best_score_)

Лучшие параметры (линейная регрессия): {'ridge__alpha': 545.5594781168514}
Лучшая оценка: 0.47293946024799427


In [15]:
# Выбираем модель на основе лучшего результата
if logistic_grid_search.best_score_ > ridge_grid_search.best_score_:
    best_pipe = logistic_grid_search.best_estimator_
    model_name = 'Логистическая регрессия'
else:
    best_pipe = ridge_grid_search.best_estimator_
    model_name = 'Линейная регрессия'

y_pred = best_pipe.predict(X_test)

print(f'Итоговые результаты классификации ({model_name}):')
print(classification_report(y_test, y_pred))
print("Границы категорий цен:", pd.qcut(data['log_price'], q=4).unique())

Итоговые результаты классификации (Логистическая регрессия):
              precision    recall  f1-score   support

           0       0.39      0.80      0.52       404
           1       0.29      0.10      0.15       459
           2       0.39      0.30      0.33       423
           3       0.67      0.61      0.64       428

    accuracy                           0.44      1714
   macro avg       0.43      0.45      0.41      1714
weighted avg       0.43      0.44      0.41      1714

Границы категорий цен: [(4.7, 5.165], (4.304, 4.7], (-0.001, 4.304], (5.165, 7.6]]
Categories (4, interval[float64, right]): [(-0.001, 4.304] < (4.304, 4.7] < (4.7, 5.165] <
                                           (5.165, 7.6]]
